## Harmony Py Library
### Job Results Iterator Example

In [1]:
import sys
sys.path.append('..')

# Install harmony-py requirements.  Not necessary if you ran `pip install harmony-py` in your kernel  
!{sys.executable} -m pip install -q -r ../requirements/core.txt

import concurrent.futures
from harmony import BBox, Client, Collection, Request, Environment


In [2]:
harmony_client = Client(env=Environment.UAT)  # assumes .netrc usage

collection = Collection(id='C1234088182-EEDTEST')
request = Request(
    collection=collection,
    spatial=BBox(-165, 52, -140, 77)
)


In [3]:
# submit an async request for processing and return the job_id
job_id = harmony_client.submit(request)
job_id


'4d3e3250-f18b-49fd-8319-5d96c8adf9b9'

In [4]:
iter = harmony_client.iterator(job_id, '/tmp/job_results_iter', True)
futures = list(map(lambda x: x['path'], iter))
(done_futures, _) = concurrent.futures.wait(futures)
for f in done_futures:
  path = f.result()
  print(path)

In [5]:
# We can iterator over the results without waiting for the job to complete before downloading
# the first results
# iter = harmony_client.iterator(job_id, '', True)
# for result in iter:
#   # print the file name
#   path_future = result['path']
#   path = path_future.result()
#   print(path)

https://harmony.uat.earthdata.nasa.gov/jobs/4d3e3250-f18b-49fd-8319-5d96c8adf9b9?linktype=https
2020_01_01_7f00ff_global_subsetted.nc.tif
2020_01_02_3200ff_global_subsetted.nc.tif
2020_01_11_7fff00_global_subsetted.nc.tif
2020_01_12_cbff00_global_subsetted.nc.tif
2020_01_13_ffe500_global_subsetted.nc.tif
2020_01_14_ff9900_global_subsetted.nc.tif
2020_01_15_ff4c00_global_subsetted.nc.tif
2020_01_16_ff0000_global_subsetted.nc.tif
2020_08_01_7f00ff_north_america_subsetted.nc.tif
2020_08_02_3200ff_north_america_subsetted.nc.tif
2020_08_03_0019ff_north_america_subsetted.nc.tif
2020_08_04_0065ff_north_america_subsetted.nc.tif
2020_01_03_0019ff_global_subsetted.nc.tif
2020_08_05_00b2ff_north_america_subsetted.nc.tif
2020_08_06_00feff_north_america_subsetted.nc.tif
2020_08_07_00ffb2_north_america_subsetted.nc.tif
2020_08_08_00ff66_north_america_subsetted.nc.tif
2020_08_09_00ff19_north_america_subsetted.nc.tif
2020_08_10_32ff00_north_america_subsetted.nc.tif
2020_08_11_7fff00_north_america_subs